<a href="https://colab.research.google.com/github/subhajitchou/RESUME_SCREENING_NLP/blob/main/Resume_Screening_Subhajit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer.six

     |████████████████████████████████| 5.6 MB 14.1 MB/s 
     |████████████████████████████████| 3.6 MB 45.9 MB/s 


### Import python modules and main pdf reader module pdfminer 

In [ ]:
import nltk
import numpy as np
from pdfminer.high_level import extract_text
import re
import subprocess
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Connect to google drive to access resumes 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
resume_path = '/content/drive/MyDrive/AI/Resume'

### Define Function to read txt from PDF 

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


def text_extract_from_pdf(pdf_path):
    return extract_text(pdf_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Define function to read names from the text extracted from PDF documents 

In [ ]:
def extract_name(txt):
    candidate_name = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'CANDIDATE':
                candidate_name.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return candidate_name

### Define function to read Phone-Numbers from the text extracted from PDF documents

In [ ]:
PHONE_NUM = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')

def phone_number_extract(resume_text):
    phone = re.findall(PHONE_NUM, resume_text)

    if phone:
        number = ''.join(phone[0])

        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

### Define function to read Email from the text extracted from PDF documents

In [ ]:
EMAIL_ID = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_email(resume_text):
    return re.findall(EMAIL_ID, resume_text)

### **Function to print name, Phone Number, email from resume**

In [ ]:
if __name__ == '__main__':
    text = text_extract_from_pdf('/content/gdrive/MyDrive/Resume_Screening/Ankit_Gupta_Resume.pdf')
    #print(text)
    names = extract_name(text)
    phone_number = phone_number_extract(text)
    emails = extract_email(text)
    if names:
        print("Name:",names[0])
    if phone_number:
        print("Phone Number:",phone_number)
    if emails:
        print("Email Address:",emails[0])

Name: Gupta Indian Institute
Phone Number: 8292858632
Email Address: ankitgupta8292@gmail.com


## **Extract Skills**

### Define function to get skills and put them in technical/ non-technical based on csv provided 

In [ ]:
# Read the csv files and prepare Technical & Non-Technical Groups

Tech_Skill_Group = []
Non_Tech_Skill_Group = []

skill_nontech = pd.read_csv('/content/drive/MyDrive/AI/Resume/nontechnicalskills.csv',header=None)
skill_tech = pd.read_csv('/content/drive/MyDrive/AI/Resume/techskill.csv',header=None)

In [ ]:
Technical_Skills_DB = nltk.flatten(skill_tech.values.tolist())

Non_Tech_Skill_Group = nltk.flatten(skill_nontech.values.tolist())
print (Non_Tech_Skill_Group)

['coldcalling', 'teamwork', 'harwork', 'creativity', 'Collaboration', 'communication', 'accounting', 'sales', 'marketing', 'dataentry', 'Leadership skills', 'Adaptability and flexibility', 'Problem-solving', 'Decision-making', 'Creativity', 'Team-working', 'TimeManagement', 'Willingness-to-learn', nan, nan, nan, nan]


In [ ]:
###  Function to extract Technical skillset
def extract_tech_skill_set(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtrd_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtrd_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtrd_tokens, 2, 3)))

    # we create a set to keep the results in.
    get_tech_skill_set = set()
    
    # we search for each token in our skills database
    for token in filtrd_tokens:
        if token.lower() in Tech_Skill:
            get_tech_skill_set.add(token)
    
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in Tech_Skill:
            get_tech_skill_set.add(ngram)
    
    return get_tech_skill_set

In [ ]:
###  Function to extract Non-Technical skillset 
def extract_nontech_skill_set(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtrd_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtrd_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtrd_tokens, 2, 3)))

    # we create a set to keep the results in.
    get_nontech_skill_set = set()

    # we search for each token in our skills database
    for token in filtrd_tokens:
      if token.lower() in Non_Tech_Skill:
        get_nontech_skill_set.add(token)

    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in Non_Tech_Skill:
            get_nontech_skill_set.add(ngram)

    return  get_nontech_skill_set

### **Function to print Technical & Non-Technical Skills from resume, and csv data provided**

In [ ]:

if __name__ == '__main__':
    tech_skill_reqd = extract_tech_skill_set(text)
    print("Technical_skills:", tech_skill_reqd )
    nontech_skills_ignored = extract_nontech_skill_set(text)
    print("NON-Technical_skills:", nontech_skills_ignored )

Technical_skills: {'Php', 'Java', 'MySQL', 'Github', 'CSS', 'newspaper', 'Framework', 'framework', 'C', 'github', 'Visual', 'PostgreSQL'}
NON-Technical_skills: set()
